# 📘 LangChain Agent를 활용한 수학 문제 해결

### 🎯 과제 목표
- LangChain AgentExecutor를 사용해 수학 문제 해결
- 자연어 질문을 받아 적절한 Tool을 순차적으로 실행하여 최종 답을 도출

### 📌 과제 내용
> "5에 7을 더한 값에 3을 곱해줘"라는 질문을 처리하는 Agent를 만들기

### 요구사항
1. Tool 정의:
   - `add(a: int, b: int)` → 덧셈
   - `multiply(a: int, b: int)` → 곱셈
2. LangChain Agent 구성
   - 위 Tool 두 개를 자동 연결하는 Agent 구현
3. 실행 결과 생성
   - 자연어로 질문 → Agent가 Tool 순서 결정 → 최종 숫자 출력


In [3]:
import os
from dotenv import load_dotenv

# OpenAI API 키 설정
load_dotenv(override=True)
MODEL = "gpt-4o-mini"


In [4]:
from langchain_core.tools import tool

# Tool 1: 덧셈
@tool
def add(a: int, b: int) -> int:
    """두 정수를 더합니다.
    
    Args:
        a: 첫 번째 정수
        b: 두 번째 정수
    
    Returns:
        두 정수의 합
    """
    print(f"[Tool 실행] add(a={a}, b={b})")
    result = a + b
    print(f"[결과] {a} + {b} = {result}")
    return result

# Tool 2: 곱셈
@tool
def multiply(a: int, b: int) -> int:
    """두 정수를 곱합니다.
    
    Args:
        a: 첫 번째 정수
        b: 두 번째 정수
    
    Returns:
        두 정수의 곱
    """
    print(f"[Tool 실행] multiply(a={a}, b={b})")
    result = a * b
    print(f"[결과] {a} × {b} = {result}")
    return result

# Tool 리스트 생성
tools = [add, multiply]

print(" Tool 정의 완료:")
print(f"  - {tools[0].name}: {tools[0].description}")
print(f"  - {tools[1].name}: {tools[1].description}")


 Tool 정의 완료:
  - add: 두 정수를 더합니다.

    Args:
        a: 첫 번째 정수
        b: 두 번째 정수

    Returns:
        두 정수의 합
  - multiply: 두 정수를 곱합니다.

    Args:
        a: 첫 번째 정수
        b: 두 번째 정수

    Returns:
        두 정수의 곱


In [ ]:
!pip install langchain-classic

In [7]:
from langchain_openai import ChatOpenAI
from langchain_classic.agents import AgentExecutor, create_openai_tools_agent
# from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_classic import hub

# 1. LLM 초기화
llm = ChatOpenAI(model=MODEL, temperature=0)

# 2. 프롬프트 가져오기 (LangChain Hub에서)
prompt = hub.pull("hwchase17/openai-functions-agent")

# 3. Agent 생성
agent = create_openai_tools_agent(llm, tools, prompt)

# 4. AgentExecutor 생성 (verbose=True로 실행 과정 확인 가능)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print(" LangChain Agent 구성 완료!")


 LangChain Agent 구성 완료!


In [8]:
# 질문 정의
question = "5에 7을 더한 값에 3을 곱해줘"

print("=" * 60)
print(f"질문: {question}")
print("=" * 60)
print("\n[Agent 실행 시작]\n")

# Agent 실행
result = agent_executor.invoke({"input": question})

print("\n" + "=" * 60)
print("[Agent 실행 완료]")
print("=" * 60)


질문: 5에 7을 더한 값에 3을 곱해줘

[Agent 실행 시작]



> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 5, 'b': 7}`


[Tool 실행] add(a=5, b=7)
[결과] 5 + 7 = 12
12
Invoking: `multiply` with `{'a': 3, 'b': 1}`


[Tool 실행] multiply(a=3, b=1)
[결과] 3 × 1 = 3
3
Invoking: `multiply` with `{'a': 12, 'b': 3}`


[Tool 실행] multiply(a=12, b=3)
[결과] 12 × 3 = 36
365에 7을 더한 값은 12이고, 그 값에 3을 곱한 결과는 36입니다.

> Finished chain.

[Agent 실행 완료]


In [9]:
# 최종 결과 출력
print("\n" + "=" * 60)
print(" 최종 결과")
print("=" * 60)
print(f"\n질문: {question}")
print(f"답변: {result['output']}")
print("\n" + "=" * 60)

# 검증: 직접 계산하여 결과 확인
expected_result = (5 + 7) * 3
print(f"\n✅ 검증: (5 + 7) × 3 = {expected_result}")
print(f"   Agent 결과와 일치하는지 확인해보세요!")



 최종 결과

질문: 5에 7을 더한 값에 3을 곱해줘
답변: 5에 7을 더한 값은 12이고, 그 값에 3을 곱한 결과는 36입니다.


✅ 검증: (5 + 7) × 3 = 36
   Agent 결과와 일치하는지 확인해보세요!


In [10]:
# 추가 테스트 케이스
test_questions = [
    "10에 5를 더한 값에 2를 곱해줘",
    "3과 4를 곱한 값에 5를 더해줘",
    "8에 2를 곱한 다음 6을 더해줘"
]

for i, test_q in enumerate(test_questions, 1):
    print(f"\n{'='*60}")
    print(f"테스트 케이스 {i}: {test_q}")
    print(f"{'='*60}")
    
    result = agent_executor.invoke({"input": test_q})
    print(f"\n결과: {result['output']}")



테스트 케이스 1: 10에 5를 더한 값에 2를 곱해줘


> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 10, 'b': 5}`


[Tool 실행] add(a=10, b=5)
[결과] 10 + 5 = 15
15
Invoking: `multiply` with `{'a': 2, 'b': 1}`


[Tool 실행] multiply(a=2, b=1)
[결과] 2 × 1 = 2
2
Invoking: `multiply` with `{'a': 15, 'b': 2}`


[Tool 실행] multiply(a=15, b=2)
[결과] 15 × 2 = 30
3010에 5를 더한 값에 2를 곱한 결과는 30입니다.

> Finished chain.

결과: 10에 5를 더한 값에 2를 곱한 결과는 30입니다.

테스트 케이스 2: 3과 4를 곱한 값에 5를 더해줘


> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'a': 3, 'b': 4}`


[Tool 실행] multiply(a=3, b=4)
[결과] 3 × 4 = 12
12
Invoking: `add` with `{'a': 3, 'b': 4}`


[Tool 실행] add(a=3, b=4)
[결과] 3 + 4 = 7
7
Invoking: `add` with `{'a': 12, 'b': 5}`


[Tool 실행] add(a=12, b=5)
[결과] 12 + 5 = 17
173과 4를 곱한 값은 12이고, 여기에 5를 더하면 17입니다.

> Finished chain.

결과: 3과 4를 곱한 값은 12이고, 여기에 5를 더하면 17입니다.

테스트 케이스 3: 8에 2를 곱한 다음 6을 더해줘


> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'a': 8, 'b': 2}`


[Tool 실행] mul